# Case study preliminaries: the vector space model

In this Notebook you will consider how to implement cosine distance to measure the similarity between text documents. You should spend around one hour on this Notebook.

In [ ]:
# Standard imports
import pandas as pd

import math

In this Notebook we will look at how to represent documents as term frequency vectors in Python, and then how to estimate the similarity of a pair of documents using cosine distance.

## Representing documents

To think about how we will represent a document in Python, let's consider two of the documents that we used in the module materials:

&nbsp;&nbsp;&nbsp;&nbsp; *John likes Mary but Mary likes Peter*

and

&nbsp;&nbsp;&nbsp;&nbsp; *Mary went to the same school as John*


The idea behind vector space models of text is that each element in a vector represents the count of words in a corresponding index of terms. So for example, if we had the term index:

    ['John', 'Mary', 'Peter', 'as', 'but', 'likes', 'same', 'school', 'the', 'to', 'went']
    
then we might represent the document *John likes Mary but Mary likes Peter* with the vector:

    [1, 2, 1, 0, 1, 2, 0, 0, 0, 0, 0]

and *Mary went to the same school as John* as:

    [1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
    

In *pandas*, a good way to store the vectors would be as a Series, in which the index of the Series is defined using the terms in the documents. We are therefore aiming to convert the sentence *John likes Mary but Mary likes Peter* into a *pandas* Series of the form:

|           |   |
|:---------:|:-:|
|**John**   |1  |
|**Mary**   |2  |
|**Peter**  |1  |
|**as**     |0  |
|**but**    |1  |
|**likes**  |2  | 
|**same**   |0  |
|**school** |0  | 
|**the**    |0  |
|**to**     |0  |
|**went**   |0  |


and *Mary went to the same school as John* as:

|           |   |
|:---------:|:-:|
|**John**   |1  |
|**Mary**   |1  |
|**Peter**  |0  |
|**as**     |1  |
|**but**    |0  |
|**likes**  |0  | 
|**same**   |1  |
|**school** |1  | 
|**the**    |1  |
|**to**     |1  |
|**went**   |1  |




We will call these vectors *term frequency vectors*, and in the rest of this Notebook we will consider how documents can easily be converted into this form, and how their similarity can then be estimated.

For the rest of this Notebook we will use the following sentences as our collection of documents:

- *John likes Mary but Mary likes Peter*

- *Mary went to the same school as John*

- *John Smith and John Jones met Mary and John Brown*

which we will store as a list:

In [ ]:
documents_ls = ['John likes Mary but Mary likes Peter',
                'Mary went to the same school as John',
                'John Smith and John Jones met Mary and John Brown']

### Tokenisation

The idea behind tokenisation is to create a sequence of the tokens which appear in a document (each *token* is an instance of a *term*). In the first case, our tokenisation mechanism will simply split a document according to the whitespace in that document. So for the document *John likes Mary but Mary likes Peter*, we require a function which will return the list of tokens:

    ['John', 'likes', 'Mary', 'but', 'Mary', 'likes', 'Peter']


Our function will be called `tokenise_document`, which takes a string of text and returns a list of the tokens in the sentence. At the moment, we will use the very simple technique of simply splitting on the whitespace in the document, for which we can use the `split` method which is defined on string objects in Python:

In [ ]:
def tokenise_document(docIn_str):
    '''Return a list of the tokens in the input string docIn_str'''
    return docIn_str.split()

To see this on one of the test sentences, try:

In [ ]:
tokenise_document(documents_ls[0])

So the tokenisation function works for the simple algorithm we are implementing at the moment. In later Notebooks we will improve this function, but it is adequate at the moment.

For most of the rest of the work in this Notebook it will be easier to deal with the tokenised sentences. We can create a list of tokenised sentences and store the list in a new variable, `tokenisedDocuments_ls`:

In [ ]:
tokenisedDocuments_ls = [tokenise_document(doc_txt) for doc_txt in documents_ls]

tokenisedDocuments_ls

So the *i*th member of the list `tokenisedDocuments_ls` is the tokenised version of the *i*th member of the list `documents_ls`.

### Building a term index

At the beginning of this Notebook we saw that the index for the term frequency vectors contains the terms in all the documents, with a count of 0 for those terms which do not appear in the document. To build a set of term frequency vectors, we therefore need to be able to identify all the terms which appear in a *collection* of documents.

As we have defined a function to carry out tokenisation, it is fairly straightforward to find the set of terms which appear in a collection of documents. We can do this by defining a function, `build_term_index`, which takes a collection of tokenised documents and returns a list of the terms which appear in the collection (although we have stored the documents as lists here, we will talk about a collection of documents in case the documents are presented as a set, a Series, or some other container type):

In [ ]:
def build_term_index(tokenisedDocuments_coll):
    '''Return a set of all the terms appearing in the 
       documents in tokenisedDocuments_coll
    '''
    allTerms_set = set()  # Store the tokens as a set to remove repetitions
    
    for tokens_coll in tokenisedDocuments_coll:
        allTerms_set = allTerms_set.union(set(tokens_coll))
        
    return list(allTerms_set)     # Return the members as a list

So, for example, to construct the index for the list of sentences stored in `tokenisedDocuments_ls`, we can call:

In [ ]:
termIndex_ls = build_term_index(tokenisedDocuments_ls)

termIndex_ls

which is the collection of terms that we want.

### Building term frequency vectors

Now that we have written a tokenisation function, we can use it to generate term frequency vectors for our documents. To find the term frequencies in a sentence, we can use the `Counter` function in the `collections` library. This returns an object which behaves like a `dict`, containing the number of occurrences of each member of a set. So to illustrate:

In [ ]:
from collections import Counter

We can use the `Counter` function to count the number of times each term appears in a tokenised sentence. So to see how many times each term appears in the first member of our list of tokenised documents:

In [ ]:
Counter(tokenisedDocuments_ls[0])

and this can be converted directly to a `pandas.Series` object, which uses the keys in the dictionary as the index terms:

In [ ]:
pd.Series(Counter(tokenisedDocuments_ls[0]))

We can now use the `reindex` method to extend the index to all the terms in the document collection.

In [ ]:
pd.Series(Counter(tokenisedDocuments_ls[0])).reindex(termIndex_ls)

As this example shows, *pandas* will create a `Series` object from a `Counter` object by filling in any new terms with `NaN` entries. So the final stage is to replace the `NaN` entries with zero using the `fillna` method:

In [ ]:
pd.Series(Counter(tokenisedDocuments_ls[0])).reindex(termIndex_ls).fillna(0)

So using these techniques we can write a function which takes a tokenised document and an index of terms, and returns a `pd.Series` object representing the term frequency vector for the document:

In [ ]:
def build_tf_vector(tokenisedDocument_ls, termIndex_ls):
    '''Return a pandas Series representing the term 
       frequency vector of the tokenised document 
       tokenisedDocument_ls, and indexed with termIndex_ls
    '''
    
    return pd.Series(Counter(tokenisedDocument_ls),
                     index=termIndex_ls).fillna(0)

For example:

In [ ]:
build_tf_vector(tokenisedDocuments_ls[0], 
                  termIndex_ls)

The function has returned the intended target representations.

## Calculating cosine distance

Now that we have created the functions needed to represent documents as term vectors, we need to be able to compare them using cosine similarity. Let's start by encoding the first two of the example documents:

In [ ]:
# First document is "John likes Mary but Mary likes Peter"
termFrequencyVector1_ss = build_tf_vector(tokenisedDocuments_ls[0],
                                   termIndex_ls)
termFrequencyVector1_ss

In [ ]:
# Second document is "Mary went to the same school as John
termFrequencyVector2_ss = build_tf_vector(tokenisedDocuments_ls[1],
                                     termIndex_ls)
termFrequencyVector2_ss

As we saw in Section 3.3 of Part 22, if ***A*** and ***B*** are term vectors representing two documents, then the cosine similarity is defined as:

$$\cos\theta = \frac{\bf{A} \cdot \bf{B}}{\lvert \bf{A} \rvert \lvert \bf{B} \rvert} $$

which returns a value between 0 (for documents which share no common tokens) and 1 (for documents which contain exactly the same tokens). This value is often converted into cosine distance by subtracting from 1, so that:

$$\text{distance}({\bf A}, {\bf B}) = 1-\cos\theta$$

so that the more similar the documents are, the smaller the distance between them. 

So if the two vectors are:

$${\bf A}=\langle a_1, a_2, a_3, \ldots , a_n \rangle$$

and

$${\bf B}=\langle b_1, b_2, b_3, \ldots , b_n \rangle$$

then

$$\text{distance}({\bf A},{\bf B})=1-\cos \theta = 1- \frac{\sum_{i=1}^n a_i b_i}{\sqrt{\sum_{i=1}^n a_i^2}\sqrt{\sum_{i=1}^n b_i^2}}$$


Now that we have represented the documents as term frequency vectors, it is reasonably straightforward to calculate the cosine distance between the two documents:

In [ ]:
1 - sum(termFrequencyVector1_ss * termFrequencyVector2_ss) / (math.sqrt(sum(termFrequencyVector1_ss*termFrequencyVector1_ss)) *
                                                              math.sqrt(sum(termFrequencyVector2_ss*termFrequencyVector2_ss)))

However, there is also a built-in function to calculate cosine distance in the `scipy.spatial.distance` library:

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
cosine(termFrequencyVector1_ss, termFrequencyVector2_ss)

As we would hope, this is the value that we obtained for this task in the module materials, and we will use the built-in function for the rest of the practical work in this part.

## Bringing it all together: estimating document similarity

We have now seen how to represent documents as vectors, and how to use cosine distance to estimate the documents' similarity.

To illustrate the technique, consider the same cases that we looked at in Exercise 22.1:
1. *John likes Mary but Mary likes Peter*
2. *Mary went to the same school as John*
3. *John Smith and John Jones met Mary and John Brown*

You were asked to calculate whether, using cosine distance, document 3 should be considered more similar to document 1 or to document 2.

We start off by creating strings to encode the three documents:

In [ ]:
doc1_str = 'John likes Mary but Mary likes Peter'
doc2_str = 'Mary went to the same school as John'
doc3_str = 'John Smith and John Jones met Mary and John Brown'

Next, use the `tokenise_document` function to convert the documents to token lists:

In [ ]:
tokenisedDoc1_ls = tokenise_document(doc1_str)
tokenisedDoc2_ls = tokenise_document(doc2_str)
tokenisedDoc3_ls = tokenise_document(doc3_str)

Next, build a term index for the documents:

In [ ]:
termIndex_ls = build_term_index([tokenisedDoc1_ls,
                                 tokenisedDoc2_ls,
                                 tokenisedDoc3_ls])

# Show the terms in the index
termIndex_ls

We can then use the term index to construct term vectors for each of the documents:

In [ ]:
termFrequencyVector1_ss = build_tf_vector(tokenisedDoc1_ls, termIndex_ls)
termFrequencyVector2_ss = build_tf_vector(tokenisedDoc2_ls, termIndex_ls)
termFrequencyVector3_ss = build_tf_vector(tokenisedDoc3_ls, termIndex_ls)

For example, look at the term vector for the first document:

In [ ]:
termFrequencyVector1_ss

Using the `scipy.spatial.distance.cosine` function, we can now calculate the cosine distance between document 1 and document 3:

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
# To calculate the cosine distance between doc 1 and doc 3:
cosine(termFrequencyVector1_ss, termFrequencyVector3_ss)

And the cosine distance between document 2 and document 3:

In [ ]:
# To calculate the cosine distance between doc 2 and doc 3:
cosine(termFrequencyVector2_ss, termFrequencyVector3_ss)

The cosine distance between document 1 and document 3 is smaller than that between document 2 and document 3, so we conclude that document 3 is more similar to document 1 than document 2. 

These results are, as we would have hoped, the same as we calculated manually in the Part 22.

## What next?

You have now completed this Notebook. Go to Notebook [`22.2 Preliminaries - building the classifier`](22.2 Preliminaries - building the classifier.ipynb).